# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [68]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. 
'''

## 2. Generate the mathematical model

In [89]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [90]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Chemical Production Planning

### Sets and Indices:

* $P = \{GCA, GCB, GCC\}$: Set of products.
* $C = \{C1, C2\}$: Set of components.

### Parameters:

* $A_c$: Available liters of component $c \in C$. ($A_{C1} = 10,000$, $A_{C2} = 15,000$)
* $D_p$: Minimum demand (in liters) for product $p \in P$. ($D_{GCA} = 6,000$, $D_{GCB} = 7,000$, $D_{GCC} = 9,000$)
* $E_c$: Critical element proportion in component $c \in C$. ($E_{C1} = 0.4$, $E_{C2} = 0.2$)
* $R_p$: Required critical element proportion in product $p \in P$. ($R_{GCA} \geq 0.3$, $R_{GCB} \leq 0.3$)
* $M_{GCC}$: Minimum ratio of C1 to C2 in product GCC. ($M_{GCC} = 0.3$)
* $S_p$: Profit per liter of product $p \in P$. ($S_{GCA} = 120$, $S_{GCB} = 135$, $S_{GCC} = 155$)

### Decision Variables:

* $x_{p,c}$: Liters of component $c \in C$ used in product $p \in P$.

### Objective Function:

Maximize total profit:

```
Maximize Z = ∑_(p∈P) ∑_(c∈C) S_p * x_{p,c}
```

### Constraints:

1. **Component Availability:** The total amount of each component used in all products cannot exceed its availability.
    ```
    ∀ c ∈ C: ∑_(p∈P) x_{p,c} ≤ A_c
    ```
2. **Minimum Demand:** The production of each product must meet the minimum demand.
    ```
    ∀ p ∈ P: ∑_(c∈C) x_{p,c} ≥ D_p
    ```
3. **Critical Element in GCA:** The proportion of the critical element in GCA must be at least 0.3.
    ```
    (∑_(c∈C) E_c * x_{GCA,c}) / (∑_(c∈C) x_{GCA,c}) ≥ R_{GCA}
    ```
4. **Critical Element in GCB:** The proportion of the critical element in GCB must be at most 0.3.
    ```
    (∑_(c∈C) E_c * x_{GCB,c}) / (∑_(c∈C) x_{GCB,c}) ≤ R_{GCB}
    ```
5. **Component Ratio in GCC:** The ratio of C1 to C2 in GCC must be at least 0.3.
    ```
    x_{GCC, C1} / x_{GCC, C2} ≥ M_{GCC}
    ```
6. **Non-negativity:** The amount of each component used in each product cannot be negative.
    ```
    ∀ p ∈ P, ∀ c ∈ C: x_{p,c} ≥ 0
    ```

This model is a linear programming (LP) problem and can be solved using various LP solvers to find the optimal production plan that maximizes the firm's profit while satisfying all the constraints.


## 3. Generate the pyomo code

In [91]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [92]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
from pyomo.environ import *

# Sets and Indices
P = ['GCA', 'GCB', 'GCC']  # Set of products
C = ['C1', 'C2']          # Set of components

# Parameters (Sample data provided as instructed)
A = {'C1': 10000, 'C2': 15000}      # Available liters of each component
D = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
E = {'C1': 0.4, 'C2': 0.2}          # Critical element proportion in each component
R = {'GCA': 0.3, 'GCB': 0.3}          # Required critical element proportion in each product (GCA >= 0.3, GCB <= 0.3)
M_GCC = 0.3                        # Minimum ratio of C1 to C2 in product GCC
S = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Model Creation
model = ConcreteModel()

# Decision Variables
model.x = Var(P, C, domain=NonNegativeReals)  # Liters of component c used in product p

# Objective Function: Maximize total profit
model.obj = Objective(expr=sum(S[p] * model.x[p, c] for p in P for c in C), sense=maximize)

# Constraints
# 1. Component Availability
model.component_availability = ConstraintList()
for c in C:
    model.component_availability.add(expr=sum(model.x[p, c] for p in P) <= A[c])

# 2. Minimum Demand
model.demand_constraint = ConstraintList()
for p in P:
    model.demand_constraint.add(expr=sum(model.x[p, c] for c in C) >= D[p])

# 3. Critical Element in GCA
model.gca_critical_element = Constraint(expr=(sum(E[c] * model.x['GCA', c] for c in C) / sum(model.x['GCA', c] for c in C)) >= R['GCA'])

# 4. Critical Element in GCB
model.gcb_critical_element = Constraint(expr=(sum(E[c] * model.x['GCB', c] for c in C) / sum(model.x['GCB', c] for c in C)) <= R['GCB'])

# 5. Component Ratio in GCC
model.gcc_ratio = Constraint(expr=model.x['GCC', 'C1'] / model.x['GCC', 'C2'] >= M_GCC)

# Solve the model
solver = SolverFactory('glpk')  # You can use any LP solver available
solver.solve(model)

# Print the optimal solution
print("Optimal Solution:")
for p in P:
    for c in C:
        print(f"x[{p}, {c}] = {model.x[p, c]():.2f}")
print(f"Total Profit = {model.obj():.2f}")
```
**Explanation and Sample Data Usage:**

*   The code defines sets, parameters, and decision variables as per the problem description.
*   Sample data is provided for parameters A, D, E, R, M\_GCC, and S as instructed.
*   The objective function and constraints are formulated using Pyomo expressions.
*   The model is solved using the 'glpk' solver (you can choose another solver if needed).
*   The optimal solution, showing the amount of each component used in each product and the total profit, is printed. 

**Remember to install Pyomo and a solver (e.g., glpk) before running the code.**


## 4. Input problem data and try running the generated code

In [95]:
from pyomo.environ import *

# Sets and Indices
P = ['GCA', 'GCB', 'GCC']  # Set of products
C = ['C1', 'C2']          # Set of components

# Parameters (Sample data provided as instructed)
A = {'C1': 10000, 'C2': 15000}      # Available liters of each component
D = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
E = {'C1': 0.4, 'C2': 0.2}          # Critical element proportion in each component
R = {'GCA': 0.3, 'GCB': 0.3}          # Required critical element proportion in each product (GCA >= 0.3, GCB <= 0.3)
M_GCC = 0.3                        # Minimum ratio of C1 to C2 in product GCC
S = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Model Creation
model = ConcreteModel()

# Decision Variables
model.x = Var(P, C, domain=NonNegativeReals)  # Liters of component c used in product p

# Objective Function: Maximize total profit
model.obj = Objective(expr=sum(S[p] * model.x[p, c] for p in P for c in C), sense=maximize)

# Constraints
# 1. Component Availability
model.component_availability = ConstraintList()
for c in C:
    model.component_availability.add(expr=sum(model.x[p, c] for p in P) <= A[c])

# 2. Minimum Demand
model.demand_constraint = ConstraintList()
for p in P:
    model.demand_constraint.add(expr=sum(model.x[p, c] for c in C) >= D[p])

# 3. Critical Element in GCA
model.gca_critical_element = Constraint(expr=(sum(E[c] * model.x['GCA', c] for c in C) / sum(model.x['GCA', c] for c in C)) >= R['GCA'])

# 4. Critical Element in GCB
model.gcb_critical_element = Constraint(expr=(sum(E[c] * model.x['GCB', c] for c in C) / sum(model.x['GCB', c] for c in C)) <= R['GCB'])

# 5. Component Ratio in GCC
model.gcc_ratio = Constraint(expr=model.x['GCC', 'C1'] / model.x['GCC', 'C2'] >= M_GCC)

# Solve the model
solver = SolverFactory('glpk')  # You can use any LP solver available
solver.solve(model)

# Print the optimal solution
print("Optimal Solution:")
for p in P:
    for c in C:
        print(f"x[{p}, {c}] = {model.x[p, c]():.2f}")
print(f"Total Profit = {model.obj():.2f}")

ValueError: Model constraint (gca_critical_element) contains nonlinear terms that cannot be written to LP format

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [82]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Sets and Indices:**

* $P = \{GCA, GCB, GCC\}$: Set of products
* $C = \{C1, C2\}$: Set of components

**Parameters:**

* $a_i$: Available liters of component $i \in C$ 
    * $a_{C1} = 10,000$
    * $a_{C2} = 15,000$
* $d_j$: Minimum demand (in liters) for product $j \in P$
    * $d_{GCA} = 6,000$
    * $d_{GCB} = 7,000$
    * $d_{GCC} = 9,000$
* $e_i$: Critical element proportion in component $i \in C$
    * $e_{C1} = 0.4$
    * $e_{C2} = 0.2$
* $l_j$: Minimum required proportion of critical element in product $j \in P$
    * $l_{GCA} = 0.3$
    * $l_{GCB} = 0.3$ (maximum proportion)
* $r_{GCC}$: Minimum ratio of C1 to C2 in product GCC
    * $r_{GCC} = 0.3$
* $p_j$: Profit per liter of product $j \in P$
    * $p_{GCA} = 120$
    * $p_{GCB} = 135$
    * $p_{GCC} = 155$

**Decision Variables:**

* $x_{ij}$: Liters of component $i \in C$ used in product $j \in P$

**Objective Function (Maximize Profit):**

```
Maximize Z = ∑

In [83]:
print(response2.text)

```python
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

# Define objective function (maximize profit)
model.profit = Objective(sum(p[j] * sum(model.x[i, j] for i in C) for j in P), sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in